## 用folium生成具有OSM底图的交互式的底图并另存为html文件

In [ ]:
# 用geopandas读取转换后的文件
import folium
import geopandas as gpd
from branca.colormap import linear
import json

import matplotlib.pyplot as plt
import numpy as np
import matplotlib

In [ ]:
# 通过geopandas读取GeoJSON文件以方便查看属性值
gdf = gpd.read_file('../Data/Transport/Cycling/GEOJSON/cycle_parking.geojson')
gdf.shape

In [ ]:
null_count = gdf['PRK_PROVIS'].isnull().sum()

print(f"PRK_PROVIS列中有 {null_count} 个空值")

In [ ]:
# 直接读取geojson文件并在osm底图上进行可视化


# 设置伦敦市中心的经纬度
london_center = [51.5074, -0.1278]

# 创建一个地图对象，中心设为伦敦市中心
m = folium.Map(location=[51.5074, -0.1278], zoom_start=13)
"""
#----------------------------------------------
# 展示连续变量
# 读取 GeoJSON 文件
with open('../Data/Transport/Cycling/GEOJSON/cycle_parking.geojson') as f:
    data = json.load(f)

# 遍历所有要素，为 'PRK_CPT' 属性的空值赋值为 0.0
for feature in data['features']:
    if feature['properties'].get('PRK_CPT') is None:
        feature['properties']['PRK_CPT'] = 0.0
        
# 创建一个线性颜色映射
# 假设连续变量的范围是从 0 到 100
colormap = linear.YlOrRd_09.scale(0, 200)
colormap.caption = '连续变量的颜色映射'
# 将颜色映射图例添加到地图
folium.GeoJson(
    data,
    name='cycle_parking', # 使用文件名作为图层名称
    style_function=lambda feature: {
        'fillColor': colormap(feature['properties'].get('PRK_CPT', 0)),
        'color': 'black',  # 线颜色
        'weight': 2,
        'fillOpacity': 0.7
    }
).add_to(m)


# 将颜色映射图例添加到地图
colormap.add_to(m)
"""
#-------------------------------------------------
# 展示分类变量
# 读取 GeoJSON 文件
with open('../Data/Transport/Cycling/GEOJSON/restricted_route.geojson') as f:
    data = json.load(f)

# 获取所有分类变量的唯一值
categories = set(feature['properties']['BOROUGH'] for feature in data['features'])

# 使用 matplotlib 生成颜色映射
colors = plt.cm.plasma(np.linspace(0, 1, len(categories)))  # 使用 plasma 颜色映射, 也可以使用其他颜色映射
color_map = {category: matplotlib.colors.to_hex(color) for category, color in zip(categories, colors)}

# 添加 GeoJSON 图层到地图，根据分类变量自动分配颜色
folium.GeoJson(
    data,
    name='geojson',
    style_function=lambda feature: {
        'fillColor': matplotlib.colors.to_hex(color_map[feature['properties']['BOROUGH']]),
        'color': 'black',  # 边界颜色
        'weight': 2,
        'fillOpacity': 0.5
    }
).add_to(m)


# 添加图层控制器
folium.LayerControl().add_to(m)


#------------------------------------------
# 在Jupyter Notebook中直接显示地图
#m
# 保存为html文件
m.save('../Data/Transport/Cycling/cycling_map.html')

## Air Monitoring Sites

In [48]:
import geopandas as gpd
import pandas as pd
import numpy as np
from shapely.geometry import Point

In [49]:
df = pd.read_csv("../Data/AirQuality/AQ_Sites.csv")

In [50]:

df = df[df['Latitude']!=0]
df = df[df['Longitude']!=0]

In [51]:
df_opening = df[df['DateClosed'].isnull()]

In [52]:
df_opening.sample(7)

,LocalAuthorityCode,LocalAuthorityName,SiteCode,SiteName,SiteType,DateOpened,DateClosed,Latitude,Longitude
203,29,Sutton,ST9,Sutton - Beddington Village,Roadside,2020-10-16 00:00:00,NaN,51.371201,-0.131949
172,24,Merton,ME7,Merton - Willow Lane Industrial Estate,Industrial,2015-08-05 00:00:00,2016-04-01 00:00:00,51.394342,-0.165161
22,3,Bexley,BQ3,Bexley 8 - Traffic(E fast),Kerbside,2007-08-14 00:00:00,2010-04-13 00:00:00,51.456855,0.194546
188,27,Richmond,RI1,Richmond Upon Thames - Castelnau,Roadside,2000-06-14 00:15:00,NaN,51.480189,-0.237335
102,13,Hammersmith and Fulham,HF1,Hammersmith and Fulham - Broadway,Roadside,1999-08-23 18:15:00,2009-01-20 00:00:00,51.492766,-0.223601
229,32,Wandsworth,WA3,Wandsworth - Roehampton,Rural,1994-10-31 00:00:00,2000-11-24 13:45:00,51.456920,-0.245562
57,8,Croydon,CR8,Croydon - Norbury Manor,Urban Background,2014-12-01 00:00:00,NaN,51.410039,-0.127523


In [55]:
geometry = [Point(xy) for xy in zip(df_opening['Longitude'], df_opening['Latitude'])]

# 创建GeoDataFrame
gdf = gpd.GeoDataFrame(df_opening, geometry=geometry)

In [56]:
gdf['geometry']

2       POINT (0.17789 51.56375)
3       POINT (0.13286 51.52939)
8       POINT (0.15891 51.49061)
10      POINT (0.13728 51.49465)
15      POINT (0.18488 51.46598)
                 ...            
238    POINT (-0.12163 51.51198)
241    POINT (-0.14711 51.49225)
242    POINT (-0.13194 51.49468)
243    POINT (-0.15459 51.52254)
245    POINT (-0.15279 51.51393)
Name: geometry, Length: 101, dtype: geometry

In [57]:
import folium

In [59]:
# 设置坐标参考系统(CRS)为英国国家网格参考系统 (EPSG:27700)，并转换为WGS84 (EPSG:4326)
gdf.crs = 'epsg:4326'
gdf = gdf.to_crs(epsg=4326)

# 创建 folium 地图对象
m = folium.Map(
    location=[gdf.geometry.y.mean(), gdf.geometry.x.mean()], # 设置地图的中心点
    zoom_start=10 # 初始缩放级别
)

# 在地图上添加点，同时检查几何对象是否为空
for _, row in gdf.iterrows():
    if row.geometry is not None and not row.geometry.is_empty:
        # 使用内置的图标
        icon = folium.Icon(
            icon='glyphicon glyphicon-map-marker',  # 使用内置的glyphicon图标，例如地图标记
            prefix='glyphicon',  # 指定图标集，这里使用Bootstrap的glyphicon
            color='red',  # 指定图标颜色
        )
        folium.Marker(
            location=[row.geometry.y, row.geometry.x],
            icon=icon,
            popup=str(row['SiteCode'])  # 假设每个点有一个 'SiteCode' 列作为标识，并确保将其转换为字符串
        ).add_to(m)

# 添加图层切换控件
folium.LayerControl().add_to(m)


m.save('../Data/AirQuality/SitesOpening.html')